In [15]:
from data_loader import DataLoader
from classes import NewFeatures, FlightPerformance, AirlineDelays, DataCleaning
from imports import *
from config import CSV_FILE_WEATHER
from config import EXCEL_FILE2

In [2]:
# Load the CSV file using the path defined in config.py
BRU2023_weather = pd.read_csv(CSV_FILE_WEATHER)

# Display the first few rows of the weather dataframe
BRU2023_weather.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Brussels,Belgium",2023-01-01,16.1,10.4,13.0,16.1,10.4,13.0,8.2,74.0,...,2.0,2,10,2023-01-01T08:45:09,2023-01-01T16:47:08,0.31,"Rain, Overcast",Cloudy skies throughout the day with rain.,rain,"E5287,06459099999,06439099999,D0152,0645009999..."
1,"Brussels,Belgium",2023-01-02,13.6,6.6,9.9,13.6,4.3,8.5,8.6,91.9,...,1.2,1,10,2023-01-02T08:45:04,2023-01-02T16:48:11,0.35,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"06459099999,06439099999,D0152,F4778,0644709999..."
2,"Brussels,Belgium",2023-01-03,8.9,3.8,6.8,5.8,0.7,3.8,5.4,91.2,...,2.2,2,10,2023-01-03T08:44:55,2023-01-03T16:49:16,0.38,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"06459099999,06439099999,D0152,F4778,0644709999..."
3,"Brussels,Belgium",2023-01-04,12.4,8.6,11.2,12.4,4.8,10.4,9.6,90.2,...,0.9,1,10,2023-01-04T08:44:43,2023-01-04T16:50:24,0.41,"Rain, Overcast",Cloudy skies throughout the day with rain.,rain,"06459099999,06439099999,D0152,F4778,0644709999..."
4,"Brussels,Belgium",2023-01-05,11.4,9.8,10.7,11.4,7.5,10.5,9.1,89.8,...,1.7,1,10,2023-01-05T08:44:27,2023-01-05T16:51:35,0.45,"Rain, Overcast",Cloudy skies throughout the day with rain.,rain,"06459099999,06439099999,D0152,06447099999,0645..."


In [40]:
xls = pd.ExcelFile(EXCEL_FILE2) # Load the Excel file using the path defined in config.py
passengers_airline = pd.read_excel(xls, sheet_name='Sheet 1', header=8) # Load the specific sheets for Arrival and Departure 2023

C:\Testing\Anaconda\envs\DTCS\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [41]:
# Drop rows with any NaN values
cleaned_df = passengers_airline.dropna().copy()
cleaned_df = cleaned_df[~cleaned_df.apply(lambda row: row.astype(str).str.contains(":").any(), axis=1)] # Drop rows where any column contains ":"

# Example: Renaming columns if necessary
cleaned_df.rename(columns={'TIME': 'AIRP_PR'}, inplace=True)
missing_values = cleaned_df.isna().sum().sum()
print(f"Number of total missing values: {missing_values}")

Number of total missing values: 0


In [43]:
airport_mapping = {
    'BRUSSELS airport': 'BRU',
    'ABU DHABI INTERNATIONAL airport': 'AUH',
    'DUBAI INTERNATIONAL airport': 'DXB',
    'WIEN-SCHWECHAT airport': 'VIE',
    'MONTREAL/PIERRE ELLIOT TRUDEAU INTL, QC airport': 'YUL',
    'GENEVA airport': 'GVA',
    'ZURICH airport': 'ZRH',
    'DOUALA/AEROPORT airport': 'DLA',
    'YAOUNDE/NSIMALEN airport': 'NSI',
    'BEIJING/CAPITAL airport': 'PEK',
    'PRAHA/RUZYNE airport': 'PRG',
    'BERLIN-BRANDENBURG airport': 'BER',
    'FRANKFURT/MAIN airport': 'FRA',
    'HAMBURG airport': 'HAM',
    'MUENCHEN airport': 'MUC',
    'KOBENHAVN/KASTRUP airport': 'CPH',
    'HURGHADA / INTL airport': 'HRG',
    'ATHINAI/ELEFTHERIOS VENIZELOS airport': 'ATH',
    'GRAN CANARIA airport': 'LPA',
    'TENERIFE SUR/REINA SOFIA airport': 'TFS',
    'ALICANTE airport': 'ALC',
    'BILBAO airport': 'BIO',
    'BARCELONA/EL PRAT airport': 'BCN',
    'ADOLFO SUAREZ MADRID-BARAJAS airport': 'MAD',
    'MALAGA/COSTA DEL SOL airport': 'AGP',
    'VALENCIA airport': 'VLC',
    'HELSINKI/VANTAA airport': 'HEL',
    'TOULOUSE/BLAGNAC airport': 'TLS',
    'LYON SAINT-EXUPERY airport': 'LYS',
    'MARSEILLE-PROVENCE airport': 'MRS',
    'NICE-COTE D\'AZUR airport': 'NCE',
    'PARIS-CHARLES DE GAULLE airport': 'CDG',
    'ACCRA/KOTOKA INTERNATIONAL airport': 'ACC',
    'BUDAPEST/LISZT FERENC INTERNATIONAL airport': 'BUD',
    'DUBLIN airport': 'DUB',
    'TEL-AVIV/BEN GURION airport': 'TLV',
    'MILANO/MALPENSA airport': 'MXP',
    'MILANO/LINATE airport': 'LIN',
    'VENEZIA/TESSERA airport': 'VCE',
    'ROMA/FIUMICINO airport': 'FCO',
    'RIGA airport': 'RIX',
    'CASABLANCA/MOHAMMED V airport': 'CMN',
    'NADOR/EL AROUI airport': 'NDR',
    'MARRAKECH/MENARA airport': 'RAK',
    'TANGER/IBN BATOUTA airport': 'TNG',
    'LUQA airport': 'MLA',
    'AMSTERDAM/SCHIPHOL airport': 'AMS',
    'OSLO/GARDERMOEN airport': 'OSL',
    'WARSZAWA/CHOPINA airport': 'WAW',
    'FARO airport': 'FAO',
    'PORTO airport': 'OPO',
    'LISBOA airport': 'LIS',
    'HAMAD INTERNATIONAL airport': 'DOH',
    'BUCURESTI/HENRI COANDA airport': 'OTP',
    'GOTEBORG/LANDVETTER airport': 'GOT',
    'STOCKHOLM/ARLANDA airport': 'ARN',
    'TUNIS/CARTHAGE airport': 'TUN',
    'DJERBA/ZARZIS airport': 'DJE',
    'ANTALYA (MIL-CIV) airport': 'AYT',
    'IZMIR/ADNAN MENDERES airport': 'ADB',
    'ISTANBUL/SABIHA GOKCEN airport': 'SAW',
    'MANCHESTER airport': 'MAN',
    'LONDON HEATHROW airport': 'LHR',
    'NEWARK LIBERTY INTERNATIONAL, NJ. airport': 'EWR',
    'WASHINGTON DULLES INTERNATIONAL, DC. airport': 'IAD',
    'NEW YORK/JOHN F. KENNEDY INTERNATIONAL, NY. airport': 'JFK',
    'CHICAGO - O\'HARE INTERNATIONAL, IL. airport': 'ORD'
}

In [45]:
def convert_to_route_code(row):
    airports = row.split(' - ')
    if len(airports) == 2:
        return f"{airport_mapping.get(airports[0].strip(), '')}-{airport_mapping.get(airports[1].strip(), '')}"
    return None

# Apply the function to the 'AIRP_PR' column
cleaned_df['route_code'] = cleaned_df['AIRP_PR'].apply(convert_to_route_code)
cleaned_df.insert(cleaned_df.columns.get_loc('AIRP_PR') + 1, 'Route_Code', cleaned_df.pop('Route_Code'))
cleaned_df.head()

,AIRP_PR,Route_Code,2023-12,2023-11,2023-10,2023-09,2023-08,2023-07,2023-06,2023-05,2023-04,2023-03,2023-02,2023-01,route_code
1,BRUSSELS airport - ABU DHABI INTERNATIONAL air...,BRU-AUH,12984,10866,11936,11512,11982,11782,10825,11002,11066,10308,10499,12327,BRU-AUH
2,BRUSSELS airport - DUBAI INTERNATIONAL airport,BRU-DXB,32680,31786,35126,32832,36292,35225,26333,31871,33002,34007,31856,26111,BRU-DXB
3,BRUSSELS airport - WIEN-SCHWECHAT airport,BRU-VIE,40498,45022,51144,50376,36363,39385,49519,51507,49228,42495,30885,27292,BRU-VIE
4,BRUSSELS airport - MONTREAL/PIERRE ELLIOT TRUD...,BRU-YUL,17177,16098,16362,27568,30923,30138,27306,20960,22070,18384,14738,19246,BRU-YUL
6,BRUSSELS airport - GENEVA airport,BRU-GVA,46107,39551,41914,39303,34961,38218,39607,43132,43469,46355,41531,40455,BRU-GVA
